In [1]:
import numpy as np

This is the input for the cube. Lay it down flat on a surface and go through the elements. Assume the first piece is a corner. Encode corner elements with 0, and straight pieces with 1.

In [2]:
cube_blue = [0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1]
cube_green = [0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1]

Specify which cube to use, and make sure it has the required number of pieces (27):

In [3]:
cube = cube_blue

In [4]:
len(cube)

27

Define functions necessary for solving:

In [5]:
# This returns the index of the first
# stack element that hasn't been determined
# (i.e. is still 0)
def get_first_0_index():
    global stack
    if 0 in stack:
        return next(x for x, val in enumerate(stack) if val == 0)
    else:
        return len(stack)

In [6]:
# If a violation occured, this returns the stack
# index where a change needs to happen to avoid
# the violation
def get_last_changeable_index():
    global stack
    first0 = get_first_0_index()
    last_changeable = [x for x, val in enumerate(stack[:first0 - 1]) if val not in [-1, 4]][-1]
    return last_changeable

In [7]:
# Helper to return possible new
# directions based on the current direction
def new_dirs(cur_dir):
    if cur_dir in [1, -1]:
        return (np.array([2, -2, 3, -3]))
    if cur_dir in [2, -2]:
        return (np.array([1, -1, 3, -3]))
    if cur_dir in [3, -3]:
        return (np.array([1, -1, 2, -2]))

In [8]:
# Tests whether a move extends the cube beyond 3x3x3
# or wants to put elements into space that is already
# occupied by other elements
def test_violation():
    global stack
    i = get_first_0_index()
    violation = False
    cur_dir = 2
    cur_pos = np.array([0,-1,0])
    max_extents = np.array([0, 0, 0])
    min_extents = np.array([0, 0, 0])
    occupied = np.empty((0,3), int)
    for j in stack[0:i]:
        index = abs(cur_dir) - 1
        factor = np.sign(cur_dir)
        # move current position
        cur_pos[index] += factor
        if j != -1:
            cur_dir = new_dirs(cur_dir)[j-1]
        # update cube extents
        max_extents = np.maximum(cur_pos, max_extents)
        min_extents = np.minimum(cur_pos, min_extents)
        total_extent = np.abs(max_extents - min_extents) + 1
        if (max(total_extent) > 3) or (any(np.equal(occupied, cur_pos).all(1))):
            violation = True
            break
        occupied = np.vstack((occupied, cur_pos))
    return(violation)

In [9]:
# Reverts the progress to the last piece
# that has options for orientation which
# have not yet been tried
def revert():
    global stack, rev
    i = get_last_changeable_index()
    print("Revert to " + str(i) + "!")
    stack[i + 1:] = np.zeros(len(stack[i+1:]), int)
    rev = True

In [10]:
# Move one step forward in solving the puzzle,
# i.e. add elements to the stack of instructions
def forward():
    global stack, cube, rev, solved
    i = get_first_0_index()
    if rev:
        i -= 1
        rev = False
    print("---- Forward " + str(i))
    if cube[i] == 1:
        stack[i] = -1
    else:
        stack[i] += 1
    print(stack)

Now solve the puzzle:

In [11]:
rev = False # Indicates whether the last step was a reversion
solved = False
stack = [0] * len(cube) # The instructions on how to solve will be stored in here
while not solved:
    forward()
    if test_violation():
        revert()
    if not (0 in stack):
        solved = True

---- Forward 0
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 1
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 2
[1, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 3
[1, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 4
[1, 1, -1, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 5
[1, 1, -1, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 3!
---- Forward 3
[1, 1, -1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 4
[1, 1, -1, 2, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 5
[1, 1, -1, 2, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 6
[1, 1, -1, 2, -1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Reve

---- Forward 20
[1, 1, -1, 2, -1, 2, 3, -1, 3, 4, 1, -1, 3, 2, 2, -1, 2, 4, -1, 2, 1, 0, 0, 0, 0, 0, 0]
Revert to 19!
---- Forward 19
[1, 1, -1, 2, -1, 2, 3, -1, 3, 4, 1, -1, 3, 2, 2, -1, 2, 4, -1, 3, 0, 0, 0, 0, 0, 0, 0]
---- Forward 20
[1, 1, -1, 2, -1, 2, 3, -1, 3, 4, 1, -1, 3, 2, 2, -1, 2, 4, -1, 3, 1, 0, 0, 0, 0, 0, 0]
Revert to 19!
---- Forward 19
[1, 1, -1, 2, -1, 2, 3, -1, 3, 4, 1, -1, 3, 2, 2, -1, 2, 4, -1, 4, 0, 0, 0, 0, 0, 0, 0]
---- Forward 20
[1, 1, -1, 2, -1, 2, 3, -1, 3, 4, 1, -1, 3, 2, 2, -1, 2, 4, -1, 4, 1, 0, 0, 0, 0, 0, 0]
Revert to 16!
---- Forward 16
[1, 1, -1, 2, -1, 2, 3, -1, 3, 4, 1, -1, 3, 2, 2, -1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 17
[1, 1, -1, 2, -1, 2, 3, -1, 3, 4, 1, -1, 3, 2, 2, -1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 16!
---- Forward 16
[1, 1, -1, 2, -1, 2, 3, -1, 3, 4, 1, -1, 3, 2, 2, -1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 17
[1, 1, -1, 2, -1, 2, 3, -1, 3, 4, 1, -1, 3, 2, 2, -1, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward

Revert to 10!
---- Forward 10
[1, 1, -1, 2, -1, 2, 4, -1, 1, 1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 11
[1, 1, -1, 2, -1, 2, 4, -1, 1, 1, 4, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 9!
---- Forward 9
[1, 1, -1, 2, -1, 2, 4, -1, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 10
[1, 1, -1, 2, -1, 2, 4, -1, 1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 11
[1, 1, -1, 2, -1, 2, 4, -1, 1, 2, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 12
[1, 1, -1, 2, -1, 2, 4, -1, 1, 2, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 10!
---- Forward 10
[1, 1, -1, 2, -1, 2, 4, -1, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 11
[1, 1, -1, 2, -1, 2, 4, -1, 1, 2, 2, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 12
[1, 1, -1, 2, -1, 2, 4, -1, 1, 2, 2, -1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 10!
---- Forward 10
[1, 1, -1, 2, -1

---- Forward 19
[1, 1, -1, 2, -1, 2, 4, -1, 3, 3, 1, -1, 4, 2, 2, -1, 3, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0]
Revert to 17!
---- Forward 17
[1, 1, -1, 2, -1, 2, 4, -1, 3, 3, 1, -1, 4, 2, 2, -1, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 18
[1, 1, -1, 2, -1, 2, 4, -1, 3, 3, 1, -1, 4, 2, 2, -1, 3, 2, -1, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 19
[1, 1, -1, 2, -1, 2, 4, -1, 3, 3, 1, -1, 4, 2, 2, -1, 3, 2, -1, 1, 0, 0, 0, 0, 0, 0, 0]
Revert to 17!
---- Forward 17
[1, 1, -1, 2, -1, 2, 4, -1, 3, 3, 1, -1, 4, 2, 2, -1, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 18
[1, 1, -1, 2, -1, 2, 4, -1, 3, 3, 1, -1, 4, 2, 2, -1, 3, 3, -1, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 19
[1, 1, -1, 2, -1, 2, 4, -1, 3, 3, 1, -1, 4, 2, 2, -1, 3, 3, -1, 1, 0, 0, 0, 0, 0, 0, 0]
Revert to 17!
---- Forward 17
[1, 1, -1, 2, -1, 2, 4, -1, 3, 3, 1, -1, 4, 2, 2, -1, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 18
[1, 1, -1, 2, -1, 2, 4, -1, 3, 3, 1, -1, 4, 2, 2, -1, 3, 4, -1, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 16!
---- Forwar

Revert to 14!
---- Forward 14
[1, 1, -1, 2, -1, 2, 4, -1, 4, 3, 1, -1, 4, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 15
[1, 1, -1, 2, -1, 2, 4, -1, 4, 3, 1, -1, 4, 3, 4, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 13!
---- Forward 13
[1, 1, -1, 2, -1, 2, 4, -1, 4, 3, 1, -1, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 14
[1, 1, -1, 2, -1, 2, 4, -1, 4, 3, 1, -1, 4, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 10!
---- Forward 10
[1, 1, -1, 2, -1, 2, 4, -1, 4, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 11
[1, 1, -1, 2, -1, 2, 4, -1, 4, 3, 2, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 10!
---- Forward 10
[1, 1, -1, 2, -1, 2, 4, -1, 4, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 11
[1, 1, -1, 2, -1, 2, 4, -1, 4, 3, 3, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 10!
---- Forward 10
[1, 1, -1, 2, -1, 2, 4, -1, 4, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 1

...

Revert to 17!
---- Forward 17
[1, 1, -1, 3, -1, 1, 2, -1, 4, 3, 2, -1, 2, 3, 3, -1, 4, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 18
[1, 1, -1, 3, -1, 1, 2, -1, 4, 3, 2, -1, 2, 3, 3, -1, 4, 3, -1, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 17!
---- Forward 17
[1, 1, -1, 3, -1, 1, 2, -1, 4, 3, 2, -1, 2, 3, 3, -1, 4, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 18
[1, 1, -1, 3, -1, 1, 2, -1, 4, 3, 2, -1, 2, 3, 3, -1, 4, 4, -1, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 14!
---- Forward 14
[1, 1, -1, 3, -1, 1, 2, -1, 4, 3, 2, -1, 2, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 15
[1, 1, -1, 3, -1, 1, 2, -1, 4, 3, 2, -1, 2, 3, 4, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 13!
---- Forward 13
[1, 1, -1, 3, -1, 1, 2, -1, 4, 3, 2, -1, 2, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 14
[1, 1, -1, 3, -1, 1, 2, -1, 4, 3, 2, -1, 2, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 12!
---- Forward 12
[1, 1, -1, 3, -1, 1, 2, -1, 4, 3, 2, -1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- 

---- Forward 17
[1, 1, -1, 3, -1, 1, 4, -1, 4, 3, 2, -1, 4, 3, 3, -1, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 18
[1, 1, -1, 3, -1, 1, 4, -1, 4, 3, 2, -1, 4, 3, 3, -1, 4, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 19
[1, 1, -1, 3, -1, 1, 4, -1, 4, 3, 2, -1, 4, 3, 3, -1, 4, 1, -1, 1, 0, 0, 0, 0, 0, 0, 0]
---- Forward 20
[1, 1, -1, 3, -1, 1, 4, -1, 4, 3, 2, -1, 4, 3, 3, -1, 4, 1, -1, 1, 1, 0, 0, 0, 0, 0, 0]
Revert to 19!
---- Forward 19
[1, 1, -1, 3, -1, 1, 4, -1, 4, 3, 2, -1, 4, 3, 3, -1, 4, 1, -1, 2, 0, 0, 0, 0, 0, 0, 0]
---- Forward 20
[1, 1, -1, 3, -1, 1, 4, -1, 4, 3, 2, -1, 4, 3, 3, -1, 4, 1, -1, 2, 1, 0, 0, 0, 0, 0, 0]
---- Forward 21
[1, 1, -1, 3, -1, 1, 4, -1, 4, 3, 2, -1, 4, 3, 3, -1, 4, 1, -1, 2, 1, 1, 0, 0, 0, 0, 0]
Revert to 20!
---- Forward 20
[1, 1, -1, 3, -1, 1, 4, -1, 4, 3, 2, -1, 4, 3, 3, -1, 4, 1, -1, 2, 2, 0, 0, 0, 0, 0, 0]
---- Forward 21
[1, 1, -1, 3, -1, 1, 4, -1, 4, 3, 2, -1, 4, 3, 3, -1, 4, 1, -1, 2, 2, 1, 0, 0, 0, 0, 0]
---- Forward 22
[1, 1, -1, 3, -1, 1, 

---- Forward 14
[1, 1, -1, 3, -1, 2, 2, -1, 2, 1, 4, -1, 1, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 12!
---- Forward 12
[1, 1, -1, 3, -1, 2, 2, -1, 2, 1, 4, -1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 13
[1, 1, -1, 3, -1, 2, 2, -1, 2, 1, 4, -1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 12!
---- Forward 12
[1, 1, -1, 3, -1, 2, 2, -1, 2, 1, 4, -1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 13
[1, 1, -1, 3, -1, 2, 2, -1, 2, 1, 4, -1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 14
[1, 1, -1, 3, -1, 2, 2, -1, 2, 1, 4, -1, 3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 15
[1, 1, -1, 3, -1, 2, 2, -1, 2, 1, 4, -1, 3, 1, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 14!
---- Forward 14
[1, 1, -1, 3, -1, 2, 2, -1, 2, 1, 4, -1, 3, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 15
[1, 1, -1, 3, -1, 2, 2, -1, 2, 1, 4, -1, 3, 1, 2, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 16
[1, 1, -1, 3, -1, 2, 2

Revert to 16!
---- Forward 16
[1, 1, -1, 3, -1, 2, 2, -1, 4, 3, 1, -1, 3, 3, 4, -1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 17
[1, 1, -1, 3, -1, 2, 2, -1, 4, 3, 1, -1, 3, 3, 4, -1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 16!
---- Forward 16
[1, 1, -1, 3, -1, 2, 2, -1, 4, 3, 1, -1, 3, 3, 4, -1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 17
[1, 1, -1, 3, -1, 2, 2, -1, 4, 3, 1, -1, 3, 3, 4, -1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 16!
---- Forward 16
[1, 1, -1, 3, -1, 2, 2, -1, 4, 3, 1, -1, 3, 3, 4, -1, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 17
[1, 1, -1, 3, -1, 2, 2, -1, 4, 3, 1, -1, 3, 3, 4, -1, 4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 18
[1, 1, -1, 3, -1, 2, 2, -1, 4, 3, 1, -1, 3, 3, 4, -1, 4, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 17!
---- Forward 17
[1, 1, -1, 3, -1, 2, 2, -1, 4, 3, 1, -1, 3, 3, 4, -1, 4, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 18
[1, 1, -1, 3, -1, 2, 2, -1, 4, 3, 1, -1, 3, 3, 4, -1, 4, 2, -1, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 17!
-

Revert to 13!
---- Forward 13
[1, 1, -1, 3, -1, 2, 4, -1, 4, 3, 1, -1, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 14
[1, 1, -1, 3, -1, 2, 4, -1, 4, 3, 1, -1, 1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 15
[1, 1, -1, 3, -1, 2, 4, -1, 4, 3, 1, -1, 1, 3, 1, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 14!
---- Forward 14
[1, 1, -1, 3, -1, 2, 4, -1, 4, 3, 1, -1, 1, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 15
[1, 1, -1, 3, -1, 2, 4, -1, 4, 3, 1, -1, 1, 3, 2, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 14!
---- Forward 14
[1, 1, -1, 3, -1, 2, 4, -1, 4, 3, 1, -1, 1, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 15
[1, 1, -1, 3, -1, 2, 4, -1, 4, 3, 1, -1, 1, 3, 3, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Revert to 14!
---- Forward 14
[1, 1, -1, 3, -1, 2, 4, -1, 4, 3, 1, -1, 1, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 15
[1, 1, -1, 3, -1, 2, 4, -1, 4, 3, 1, -1, 1, 3, 4, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
---- Forward 16
[1, 1,

Convert ```stack``` to readable instructions:

In [12]:
# Converts a direction into a readable string
def dir_to_str(dir):
    strs = ["-z", "-y", "-x", "", "x", "y", "z"]
    return(strs[dir + 3])

In [13]:
instrns = []
cur_dir = 2
for i, j in enumerate(stack):
    index = abs(cur_dir) - 1
    factor = np.sign(cur_dir)
    if j != -1:
        cur_dir = new_dirs(cur_dir)[j-1]
        instrns += [dir_to_str(cur_dir)]
    else:
        instrns += ["-"]

Now the integer instructions saved in ```stack``` are transformed to readable instructions (in ```instrns```):

This is assuming a 3d coordinate system with directions +x, +y, +z and -x, -y and -z.
Follow instructions in the list one by one.

'-' means the element goes straight, everything else means turn the snake into the
respective direction.

In [14]:
print(stack)
print("\n")
print(instrns)

[1, 1, -1, 3, -1, 2, 4, -1, 4, 3, 1, -1, 1, 4, 4, -1, 3, 2, -1, 3, 3, 1, 2, 1, 1, -1, -1]


['x', 'y', '-', 'z', '-', '-x', '-z', '-', '-y', 'z', 'x', '-', 'y', '-z', '-y', '-', 'z', '-x', '-', 'z', 'y', 'x', '-y', 'x', 'y', '-', '-']
